In [1]:
using JSON

# Specify the file name and location
filename = "tiny.json"

# Read the JSON file
data = JSON.parsefile(filename)

# Extract the details from the JSON
# For example, extract all shops
linear_model = true  # Set to false if you don't want a linear model
shops = data["shops"]
println("Shops:")
println(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]

for vehicle in vehicles
    if linear_model
        # Set all vehicles to regular if linear_model is true
        println("Vehicle Type: regular")
    else
        # Otherwise, print the actual vehicle type
        println("Vehicle ID: ", vehicle["id"], ", Type: ", vehicle["type"])
    end
end
for constraint in constraints
    println("Constraint: ", constraint)
end
for parameter in parameters
    println("Parameter: ", parameter)
end
for shop in shops
    println("Shop: ", shop)
end



Shops:
Any[Dict{String, Any}("name" => "body", "resequencing_lag" => 1), Dict{String, Any}("name" => "paint", "resequencing_lag" => 2), Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)]
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Constraint: Dict{String, Any}("max_vehicles" => 4, "cost" => 2, "min_vehicles" => 2, "id" => 1, "vehicles" => Any[1, 2, 4], "type" => "batch_size", "shop" => "body")
Constraint: Dict{String, Any}("cost" => 3, "partition" => Any[Any[1, 2], Any[3, 4, 5]], "id" => 2, "type" => "lot_change", "shop" => "paint")
Constraint: Dict{String, Any}("max_vehicles" => 2, "cost" => 2, "id" => 3, "window_size" => 3, "vehicles" => Any[1, 2, 5], "type" => "rolling_window", "shop" => "assembly")
Constraint: Dict{String, Any}("max_vehicles" => 3, "cost" => 3, "min_vehicles" => 2, "id" => 4, "vehicles" => Any[1, 2, 3], "type" => "batch_size", "shop" => "assembly")
Constraint: Dict{String, Any}("max_ve

In [2]:
resequencing_cost = parameters["resequencing_cost"]
number_of_shops = length(shops)    

println("Number of shops: ", number_of_shops)
number_of_vehicles = length(vehicles)
println("number_of_vehicles, ",number_of_vehicles)

Number of shops: 3
number_of_vehicles, 5


Here is an example of how an MILP solver works in Julia:

**Problem:**

Maximize 

$$
60x_1 + 100x_2 + 120x_3
$$

subject to 

$$
10x_1 + 20x_2 + 30x_3 \leq 50
$$

$$
x_1, x_2, x_3 \in \{0, 1\}
$$


In [16]:
using JuMP
using GLPK
model = Model(GLPK.Optimizer)  # Create the model with a solver

# Initialize an empty dictionary to store the mapping
shop_map = Dict{String, Vector{Any}}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """

# Populate the shop_map with shop names as keys and empty arrays as values
for shop in shops
    shop_map[shop["name"]] = []
end
n = number_of_shops
println("n: ", n)
# Initialize counters or variables if needed

# Assume `model` is defined somewhere before this loop (like in JuMP)
# Example: model = Model(Ipopt.Optimizer)

# Iterate through constraints and associate them with the corresponding shop in shop_map
for constraint in constraints
    name = constraint["shop"]
    if haskey(shop_map, name)
        push!(shop_map[name], constraint)
    else
        println("Warning: Shop name $name in constraints does not exist in shops.")
    end
end

# Print the resulting map in a structured format
println("Shop Map:")
for (shop, constraints) in shop_map
    a = 0
    b = 0
    c = 0
    println("Shop: $shop")
    println("  Constraints:")
    for constraint in constraints
        println("    $constraint")
        println("    Shop: ", constraint["shop"])
        if constraint["type"] == "lot_change" 
            println("    Type: lot_change")
            # Define necessary variables for lot_change
            a = a + 1
            print("a :", a)
            k = length(constraint["partition"])
            u = @variable(model, [1:n, 1:k], Bin)
            f = @variable(model, [1:n], Bin)
            
        end 
        
        if constraint["type"] == "rolling_window"
            println("    Type: lot_consistency")
            b += 1  # Increment the lot_consistency counter
            # Ensure model is defined before this
            print("b :", b)
            q = @variable(model, [1:n], Bin)
            z = @variable(model, [1:n], Bin)
        end
        
        if constraint["type"] == "batch_size"
            c += 1  # Increment the batch_size counter
            println("c :",c)
            # Define the binary variables correctly
            p  = @variable(model, [1:n], Bin)
            m = @variable(model, [1:n, 1:n], Bin)
            l = @variable(model, [1:n, 1:n], Bin)
            j = @variable(model, [1:n, 1:n], Bin)
            #I used j instead of n because n is already used.
            g = @variable(model, [1:n, 1:n], Bin)
        end
    end
end


n: 3
Shop Map:
Shop: body
  Constraints:
    Dict{String, Any}("max_vehicles" => 4, "cost" => 2, "min_vehicles" => 2, "id" => 1, "vehicles" => Any[1, 2, 4], "type" => "batch_size", "shop" => "body")
    Shop: body
c :1
Shop: assembly
  Constraints:
    Dict{String, Any}("max_vehicles" => 2, "cost" => 2, "id" => 3, "window_size" => 3, "vehicles" => Any[1, 2, 5], "type" => "rolling_window", "shop" => "assembly")
    Shop: assembly
    Type: lot_consistency
b :1    Dict{String, Any}("max_vehicles" => 3, "cost" => 3, "min_vehicles" => 2, "id" => 4, "vehicles" => Any[1, 2, 3], "type" => "batch_size", "shop" => "assembly")
    Shop: assembly
c :1
    Dict{String, Any}("max_vehicles" => 2, "cost" => 3, "min_vehicles" => 1, "id" => 5, "vehicles" => Any[4, 5], "type" => "batch_size", "shop" => "assembly")
    Shop: assembly
c :2
Shop: paint
  Constraints:
    Dict{String, Any}("cost" => 3, "partition" => Any[Any[1, 2], Any[3, 4, 5]], "id" => 2, "type" => "lot_change", "shop" => "paint")
    Sho